In [ ]:
#hide
#missing
!git clone https://github.com/marcomatteo/steel_segmentation.git

In [ ]:
#hide
#missing
!pip install -e steel_segmentation

# Visualize results

> An interactive notebook with IPywidgets to visualize model predictions and groundtruth.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcomatteo/steel_segmentation/blob/master/dev_nbs/09_visualize.ipynb)

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from IPython.display import Image

from steel_segmentation.all import *
from fastcore.all import L

import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2

## Display utilities

In [ ]:
def submission_preproccessing(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process a submission DataFrame and 
    splits the ImageId_ClassId column into ImageId and ClassId.
    """
    if not ("ImageId_ClassId" in df.columns):
        raise ValueError("Missing ImageId_ClassId column")
        
    splitted_cols = df["ImageId_ClassId"].str.split("_", expand=True)
    df["ImageId"], df["ClassId"] = splitted_cols[0], splitted_cols[1].astype("int64")
    
    return df

In [ ]:
def read_submission(file_name:str = "ensemble_submission.csv") -> pd.DataFrame:
    """
    Open a submission csv file in the `sub_path` directory and
    return the DataFrame preprocessed with `submission_preproccessing`.
    """
    df = pd.read_csv(sub_path / file_name)
    return submission_preproccessing(df)

In [ ]:
file = "ensemble_submission.csv"
file_df = read_submission(file)
file_df.head()

,ImageId_ClassId,EncodedPixels,ImageId,ClassId
0,0000f269f.jpg_1,NaN,0000f269f.jpg,1
1,0000f269f.jpg_2,NaN,0000f269f.jpg,2
2,0000f269f.jpg_3,146418 5 146424 1 146577 30 146608 1 146610 1 146612 21 146638 48 146732 69 146802 1 146808 1 146810 1 146812 1 146820 125 146962 1 146976 225 147208 249 147461 252 147716 253 147971 254 148227 254 148483 254 148739 254 148995 254 149251 254 149507 254 149763 254 150019 254 150275 254 150532 253 150788 253 151045 252 151303 250 151563 246 151848 11 151862 13 151876 189 152152 169 152430 29 152489 1 152491 1 152493 6 152500 7 152557 12,0000f269f.jpg,3
3,0000f269f.jpg_4,NaN,0000f269f.jpg,4
4,000ccc2ac.jpg_1,NaN,000ccc2ac.jpg,1


Display a sample of 5 images from `train_path`:

In [ ]:
sample_images_from_path = L(os.listdir(str(train_path))).shuffle()[:5]

@interact_manual 
def show_images(file=sample_images_from_path):
    display(Image(train_path/file))

interactive(children=(Dropdown(description='file', options=('33653ce7d.jpg', 'ec72d1ebd.jpg', '16d9c0a30.jpg',…

## Display a test submission

Per visualizzare un campione di 5 immagini, selezionare dall'elenco a discesa la *submission* e successivamente la *ClassId* e infine il nome dell'immagine.

In [ ]:
subs =[file for file in os.listdir(str(sub_path)) if file.endswith(".csv")]

def read_sub_file_from_list(sub = subs) -> pd.DataFrame:
    df = read_submission(sub)
    return submission_preproccessing(df)

def select_classid(ClassId=[1,2,3,4], n=5, shuffle=True):
    df = submission.result
    cond_NaN = df.EncodedPixels.notna()
    cond_ClassId = df['ClassId'] == ClassId
    result = df.loc[cond_NaN & cond_ClassId]
    if shuffle: return result.sample(n=n, replace=True)
    else: return result.head(n)

def update_imgids(*args):
    images.value = class_selection.result.loc[cond].ImageId.to_list()

In [ ]:
# missing
submission = interactive(read_sub_file_from_list)
display(submission)

interactive(children=(Dropdown(description='sub', options=('resnet34_fastai_sub.csv', 'ensemble_submission.csv…

In [ ]:
# missing
submission.observe(update_imgids, 'value')

class_selection = interactive(select_classid)
int_slider = class_selection.children[1]
int_slider.min = 0
int_slider.max = 50

class_selection

interactive(children=(Dropdown(description='ClassId', options=(1, 2, 3, 4), value=1), IntSlider(value=5, descr…

In [ ]:
# missing
df = class_selection.result
images = widgets.Dropdown(options=df.ImageId.to_list())

# Show the images
def show_images(file):
    row = df.loc[df["ImageId"] == file].iloc[0]
    rle = row["EncodedPixels"]
    classid = row["ClassId"]
    
    h, w = (256, 1600)
    masks = np.zeros((h, w, 4), dtype=np.float32) # 4:class 1～4 (ch:0～3)
    masks[:, :, classid-1] = rle2mask(rle=rle, value=1, shape=(h,w))
    
    image = cv2.imread(str(test_path/file))
    title = f"Image {file} with defect type: {classid}"
    plot_mask_image(name=title, img=image, mask=masks)

sub_img = interactive(show_images, file=images)
sub_img

interactive(children=(Dropdown(description='file', options=('09a71e9c9.jpg', '88afdeeb8.jpg', '88afdeeb8.jpg',…

## Compare

Per visualizzare invece immagini dal training o test set:

In [ ]:
# missing
# Create widgets
directory = widgets.Dropdown(options=[str(train_path), str(test_path)])
images = widgets.Dropdown(options=os.listdir(directory.value))

# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(directory.value)
# Tie the image options to directory value
directory.observe(update_images, 'value')

# Show the images
def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))

img = interactive(show_images, fdir=directory, file=images)
display(img)

interactive(children=(Dropdown(description='fdir', options=('../data/train_images', '../data/test_images'), va…

In [ ]:
# missing
def show_images(file):
    plot_defected_image(Path(train_path)/file)

img = interactive(show_images, file=images)
display(img)

interactive(children=(Dropdown(description='file', options=('054190eb2.jpg', '73aa2aa79.jpg', '69f87ba66.jpg',…

In [ ]:
def select_valid_classid(ClassId=[1,2,3,4], n=5, shuffle=False):
    df = read_sub_file_from_list("ensemble_validation.csv")
    
    cond_NaN = df.EncodedPixels.notna()
    cond_ClassId = df['ClassId'] == ClassId
    result = df.loc[cond_NaN & cond_ClassId]
    if shuffle: return result.sample(n=n, replace=True)
    else: return result.head(n)    

In [ ]:
# missing
def update_imgids(*args):
    images.value = class_selection.result.loc[cond].ImageId.to_list()

In [ ]:
# missing
# Tie the image options to submission value
submission.observe(update_imgids, 'value')

class_selection = interactive(select_valid_classid)
int_slider = class_selection.children[1]
int_slider.min = 0
int_slider.max = 50
display(class_selection)

interactive(children=(Dropdown(description='ClassId', options=(1, 2, 3, 4), value=1), IntSlider(value=5, descr…

In [ ]:
# missing
df = class_selection.result
images = widgets.Dropdown(options=df.ImageId.to_list())

# Show the images
def show_images(file):
    row = df.loc[df["ImageId"] == file].iloc[0]
    rle = row["EncodedPixels"]
    classid = row["ClassId"]
    
    h, w = (256, 1600)
    masks = np.zeros((h, w, 4), dtype=np.float32) # 4:class 1～4 (ch:0～3)
    masks[:, :, classid-1] = rle2mask(rle=rle, value=1, shape=(h,w))
    
    image = cv2.imread(str(train_path/file)).astype(np.uint8)
    title = f"Detected: Image {file} with defect type {classid}"
    plot_mask_image(name=title, img=image, mask=masks)
    
    plot_defected_image(train_path/file, classid)

sub_img = interactive(show_images, file=images)
display(sub_img)

interactive(children=(Dropdown(description='file', options=('0002cc93b.jpg', '01053d28f.jpg', '012a9a4c7.jpg',…